<a href="https://colab.research.google.com/github/sharath507/MINOR_PROJECT_5TH_SEM_NIE/blob/main/final_working__minor_project_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
! pip install torch
! pip install git+https://github.com/huggingface/transformers
! pip install git+https://github.com/huggingface/accelerate
! pip install qwen-vl-utils
!pip install Pillow
!pip install pandas

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-wuucpx0t
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-wuucpx0t
  Resolved https://github.com/huggingface/transformers to commit 8f38f58f3de5a35f9b8505e9b48985dce5470985
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.48.0.dev0-py3-none-any.whl size=10290514 sha256=df8f3e78d649f21920de4a6cfaa667870c5316a7a022899157ab56437415cb3b
  Stored in directory: /tmp/pip-ephem-wheel-cache-n2irfmlg/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.1
    Uninstalling transformers-4.47.1:
      Successfully uninstalled transformers-4.47.1
  Cloning https://github.com/huggingfac

In [12]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
import torch

model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct", torch_dtype=torch.float32, device_map="auto"
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/56.4k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/429M [00:00<?, ?B/s]

`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/272 [00:00<?, ?B/s]

In [13]:
# You can set min_pixels and max_pixels according to your needs, such as a token range of 256-1280, to balance performance and cost.
min_pixels = 256*28*28
max_pixels = 1280*28*28
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct", min_pixels=min_pixels, max_pixels=max_pixels)

preprocessor_config.json:   0%|          | 0.00/347 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

In [14]:
entity_unit_map = {
    'width': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'depth': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'height': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'item_weight': {'gram',
        'kilogram',
        'microgram',
        'milligram',
        'ounce',
        'pound',
        'ton'},
    'maximum_weight_recommendation': {'gram',
        'kilogram',
        'microgram',
        'milligram',
        'ounce',
        'pound',
        'ton'},
    'voltage': {'kilovolt', 'millivolt', 'volt'},
    'wattage': {'kilowatt', 'watt'},
    'item_volume': {'centilitre',
        'cubic foot',
        'cubic inch',
        'cup',
        'decilitre',
        'fluid ounce',
        'gallon',
        'imperial gallon',
        'litre',
        'microlitre',
        'millilitre',
        'pint',
        'quart'}
}

In [15]:
import re

def convert_text(received_text):
    # Define the expanded entity unit map
    expanded_entity_unit_map = {
        'width': {
            'centimetre': ['cm', 'centimeter', 'centimetre', 'centi', 'CM'],
            'foot': ['ft', 'foot', 'FT', 'Feet', 'feet'],
            'inch': ['in', 'inch', 'inches', 'IN'],
            'metre': ['m', 'meter', 'metre', 'M'],
            'millimetre': ['mm', 'millimeter', 'millimetre', 'MM'],
            'yard': ['yd', 'yard', 'Yard', 'YD']
        },
        'depth': {
            'centimetre': ['cm', 'centimeter', 'centimetre', 'centi', 'CM'],
            'foot': ['ft', 'foot', 'FT', 'Feet', 'feet'],
            'inch': ['in', 'inch', 'inches', 'IN'],
            'metre': ['m', 'meter', 'metre', 'M'],
            'millimetre': ['mm', 'millimeter', 'millimetre', 'MM'],
            'yard': ['yd', 'yard', 'Yard', 'YD']
        },
        'height': {
            'centimetre': ['cm', 'centimeter', 'centimetre', 'centi', 'CM'],
            'foot': ['ft', 'foot', 'FT', 'Feet', 'feet'],
            'inch': ['in', 'inch', 'inches', 'IN'],
            'metre': ['m', 'meter', 'metre', 'M'],
            'millimetre': ['mm', 'millimeter', 'millimetre', 'MM'],
            'yard': ['yd', 'yard', 'Yard', 'YD']
        },
        'item_weight': {
            'gram': ['g', 'gm', 'gms', 'grams', 'gram', 'G', 'Gr', 'GRM', 'grms', 'grm'],
            'kilogram': ['kg', 'kgs', 'kilogram', 'kilograms', 'KG', 'kilo'],
            'microgram': ['mcg', 'μg', 'microgram', 'micrograms', 'MG'],
            'milligram': ['mg', 'milligram', 'milligrams', 'MG'],
            'ounce': ['oz', 'ounce', 'ounces', 'OZ'],
            'pound': ['lb', 'lbs', 'pound', 'pounds', 'LB'],
            'ton': ['t', 'ton', 'tons', 'T']
        },
        'maximum_weight_recommendation': {
            'gram': ['g', 'gm', 'gms', 'grams', 'gram', 'G', 'Gr', 'GRM', 'grms', 'grm'],
            'kilogram': ['kg', 'kgs', 'kilogram', 'kilograms', 'KG', 'kilo'],
            'microgram': ['mcg', 'μg', 'microgram', 'micrograms', 'MG'],
            'milligram': ['mg', 'milligram', 'milligrams', 'MG'],
            'ounce': ['oz', 'ounce', 'ounces', 'OZ'],
            'pound': ['lb', 'lbs', 'pound', 'pounds', 'LB'],
            'ton': ['t', 'ton', 'tons', 'T']
        },
        'voltage': {
            'kilovolt': ['kV', 'kilovolt', 'kilovolts', 'KV'],
            'millivolt': ['mV', 'millivolt', 'millivolts', 'MV'],
            'volt': ['V', 'volt', 'volts']
        },
        'wattage': {
            'kilowatt': ['kW', 'kilowatt', 'kilowatts', 'KW'],
            'watt': ['W', 'watt', 'watts', 'WATT']
        },
        'item_volume': {
            'centilitre': ['cl', 'centilitre', 'centiliters', 'CL'],
            'cubic foot': ['cu ft', 'cubic foot', 'cubic feet', 'CF'],
            'cubic inch': ['cu in', 'cubic inch', 'cubic inches', 'CI'],
            'cup': ['cup', 'cups', 'C'],
            'decilitre': ['dl', 'decilitre', 'deciliters', 'DL'],
            'fluid ounce': ['fl oz', 'fluid ounce', 'fluid ounces', 'FLOZ'],
            'gallon': ['gal', 'gallon', 'gallons', 'GAL'],
            'imperial gallon': ['imp gal', 'imperial gallon', 'imperial gallons'],
            'litre': ['l', 'litre', 'liter', 'litres', 'L'],
            'microlitre': ['μl', 'microlitre', 'microliters', 'MICROL'],
            'millilitre': ['ml', 'millilitre', 'milliliter', 'milliliters', 'ML'],
            'pint': ['pt', 'pint', 'pints', 'PT'],
            'quart': ['qt', 'quart', 'quarts', 'QT']
        }
    }
    # Remove curly braces and extra spaces
    received_text = received_text.strip('{} ').strip()

    # Extract value and unit using regex
    match = re.match(r'(\d+(\.\d+)?)\s*(\w+)', received_text)
    if not match:
        return ""

    value, unit = match.group(1), match.group(3)

    # Find the correct unit from the expanded_entity_unit_map
    for entity, units in expanded_entity_unit_map.items():
        for key, aliases in units.items():
            if unit in aliases:
                return f"{value} {key}"

    return ""

In [16]:
import pandas as pd
import requests
from PIL import Image, ImageOps
from io import BytesIO
import torch




# image = ImageOps.grayscale(image)

def run_inference(image, entity_name, possible_units):
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "image": image,
                },
                {
                    "type": "text",
                    "text": f"""Can you tell me the {entity_name} of this product given in the image. Just return the
                     answer strictly in the format - {{value unit}} for example {{10.0 kilogram}}. Don't add anything
                     else in the response. Here are all the {{'possible_units': {possible_units}}}, In the output the
                     spelling of the unit must exactly match with one of the units given in the possible_units map, if not then convert it
                     to match one of the units in possible_units , the
                      value of which you detected from the image. If you can't find it, just return an empty string""",
                },
            ],
        }
    ]

    # Prepare inputs
    text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    image_inputs, video_inputs = process_vision_info(messages)
    inputs = processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    )
    inputs = inputs.to("cuda")

    try:
        with torch.no_grad():
            generated_ids = model.generate(**inputs, max_new_tokens=128)
            generated_ids_trimmed = [
                out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
            ]
            output_text = processor.batch_decode(
                generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
            )
            return output_text[0]
    except Exception as e:
        print(f"Inference failed: {e}")
        return None


In [17]:
from PIL import Image
import numpy as np
import pandas as pd


In [31]:
def process_dataframe():
  csv_path = input("enter the path of the csv file: ")
  test_df = pd.read_csv(csv_path)
  print(test_df.head())
  output_df = test_df

  url = input("enter the column name of the url: ")
  entity_type = input("enter the column name of the entity type: ")
  index_1 = input("enter the column name of the index: ")

  output_folder_path = input("enter the output folder path: ")
  for index, row in test_df.iterrows():
      image_url = row[url]
      entity_name = row[entity_type]
      corresponding_index = row[index_1]

      # Check if entity_name is valid
      if entity_name not in entity_unit_map:
          print(f"Invalid entity name: {entity_name}")
          output_df.at[index, "output"] = np.nan
          continue
      possible_units = entity_unit_map[entity_name]
      # Download image
      # image = download_image(image_url, grayscale=False)
      image = Image.open(image_url)
      if image is None:
          output_df.at[index, "output"] = np.nan
          continue

      # Run inference with timeout
      output = convert_text(run_inference(image, entity_name, possible_units))
      print(f"Completed index {corresponding_index} - total {index} - output: {output}")
      output_df.at[index, "output"] = output

        # Save to CSV every 10th interval

  try:
      output_df.to_csv(output_folder_path, index=False)
      print("output file is exported")
  except Exception as e:
      print(f"Error saving to CSV: {e}")


In [19]:
# prompt: give the code to store the image in the variable which the image is in the local folder

from PIL import Image

image_path = input("Enter the path to the image file: ") #@param {type:"string"}
try:
  image = Image.open(image_path)
  print("Image loaded successfully.")
except FileNotFoundError:
  print(f"Error: Image file not found at {image_path}")
except Exception as e:
  print(f"An error occurred: {e}")

KeyboardInterrupt: Interrupted by user

In [ ]:
# prompt: give the code to store the image url in the csv format which is inside the folder
# the csv format should contain the columns below
# slno,image_id,image_path,entity_type
# the method should be in the below format the value in the entity_type column should be filled with  the entity_type value stored in that variable
# def folder_to_csv(folder_path,entity_type):

import os
import csv

def folder_to_csv(folder_path, entity_type):
    """
    Stores image URLs from a folder into a CSV file.

    Args:
        folder_path: The path to the folder containing the images.
        entity_type: The value to be filled in the 'entity_type' column.
    """

    csv_file_path = 'image_data.csv'  # Name of the output CSV file

    with open(csv_file_path, 'w', newline='') as csvfile:
        fieldnames = ['slno', 'image_id', 'image_path', 'entity_type']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()  # Write the header row

        slno = 1
        image_id =1
        for filename in os.listdir(folder_path):
            if os.path.isfile(os.path.join(folder_path, filename)):
                # image_id = filename  # Use filename as image_id
                image_path = os.path.join(folder_path, filename) # Store the full path

                writer.writerow({
                    'slno': slno,
                    'image_id': image_id,
                    'image_path': image_path,
                    'entity_type': entity_type
                })
                slno += 1
                image_id +=1

# Example usage:
folder_path = '/content/product_images'  # Replace with the actual folder path
entity_type_value = 'item_weight'  # Replace with the desired entity type

folder_to_csv(folder_path, entity_type_value)

In [32]:
process_dataframe()


enter the path of the csv file: /content/image_data.csv
   slno  image_id                                image_path  entity_type
0     1         1  /content/product_images/download (3).jpg  item_weight
1     2         2  /content/product_images/download (2).jpg  item_weight
2     3         3  /content/product_images/download (1).jpg  item_weight
3     4         4    /content/product_images/images (1).jpg  item_weight
4     5         5      /content/product_images/download.jpg  item_weight
enter the column name of the url: image_path
enter the column name of the entity type: entity_type
enter the column name of the index: image_id
enter the output folder path: final.csv
Completed index 1 - total 0 - output: 10.0 milligram
Completed index 2 - total 1 - output: 800 milligram
Completed index 3 - total 2 - output: 800 milligram
Completed index 4 - total 3 - output: 2.0 gram
Completed index 5 - total 4 - output: 400 milligram
Completed index 6 - total 5 - output: 10.0 kilogram
Completed inde

In [33]:
# prompt: give the code to display the csv file

import pandas as pd

def display_csv(file_path):
  """Displays the contents of a CSV file as a pandas DataFrame.

  Args:
    file_path: The path to the CSV file.
  """
  try:
    df = pd.read_csv(file_path)
    print(df)
  except FileNotFoundError:
    print(f"Error: File not found at {file_path}")
  except pd.errors.EmptyDataError:
    print(f"Error: The CSV file at {file_path} is empty.")
  except pd.errors.ParserError:
    print(f"Error: Could not parse the CSV file at {file_path}. Check its format.")
  except Exception as e:
    print(f"An error occurred: {e}")


# Example usage (assuming the CSV is named 'image_data.csv' in the current directory)
display_csv('final.csv')

   slno  image_id                                image_path  entity_type  \
0     1         1  /content/product_images/download (3).jpg  item_weight   
1     2         2  /content/product_images/download (2).jpg  item_weight   
2     3         3  /content/product_images/download (1).jpg  item_weight   
3     4         4    /content/product_images/images (1).jpg  item_weight   
4     5         5      /content/product_images/download.jpg  item_weight   
5     6         6    /content/product_images/images (2).jpg  item_weight   
6     7         7        /content/product_images/images.jpg  item_weight   

           output  
0  10.0 milligram  
1   800 milligram  
2   800 milligram  
3        2.0 gram  
4   400 milligram  
5   10.0 kilogram  
6        2.0 gram  


In [36]:
# prompt: give the command to run model.py

!python /content/qwen_model/model.py


  File "/content/qwen_model/model.py", line 1
    ! pip install torch
    ^
SyntaxError: invalid syntax


In [38]:
from qwen_model import model as md


In [39]:
md.process_dataframe()

enter the path of the csv file: /content/image_data.csv
   slno  image_id                                image_path  entity_type
0     1         1  /content/product_images/download (3).jpg  item_weight
1     2         2  /content/product_images/download (2).jpg  item_weight
2     3         3  /content/product_images/download (1).jpg  item_weight
3     4         4    /content/product_images/images (1).jpg  item_weight
4     5         5      /content/product_images/download.jpg  item_weight
enter the column name of the url: image_path
enter the column name of the entity type: entity_type
enter the column name of the index: image_id
enter the output folder path: final01.csv
Completed index 1 - total 0 - output: 10.0 milligram
Completed index 2 - total 1 - output: 800 milligram
Completed index 3 - total 2 - output: 800 milligram
Completed index 4 - total 3 - output: 2.0 gram
Completed index 5 - total 4 - output: 400 milligram
Completed index 6 - total 5 - output: 10.0 kilogram
Completed in

In [ ]:
import pandas as pd

def calculate_accuracy_and_precision(actual_file, predicted_file, actual_column='actual', predicted_column='predicted'):
    # Load the actual and predicted data from the CSV files
    actual_data = pd.read_csv(actual_file)
    predicted_data = pd.read_csv(predicted_file)

    # Print the column names for debugging
    print("Actual Data Columns:", actual_data.columns.tolist())
    print("Predicted Data Columns:", predicted_data.columns.tolist())

    # Ensure that the actual and predicted data have the same length
    if len(actual_data) != len(predicted_data):
        raise ValueError("The actual and predicted data must have the same number of instances.")

    # Extract actual and predicted values
    actual = actual_data[actual_column]
    predicted = predicted_data[predicted_column]

    # Calculate True Positives, True Negatives, False Positives, and False Negatives
    true_positives = ((actual == 1) & (predicted == 1)).sum()
    true_negatives = ((actual == 0) & (predicted == 0)).sum()
    false_positives = ((actual == 0) & (predicted == 1)).sum()
    false_negatives = ((actual == 1) & (predicted == 0)).sum()

    # Calculate accuracy
    total_instances = len(actual)
    accuracy = (true_positives + true_negatives) / total_instances * 100

    # Calculate precision
    if (true_positives + false_positives) > 0:
        precision = true_positives / (true_positives + false_positives) * 100
    else:
        precision = 0.0  # Avoid division by zero

    actual_data.columns = actual_data.columns.str.strip()  # Remove leading/trailing spaces
    predicted_data.columns = predicted_data.columns.str.strip()
    return accuracy, precision

# Example usage:
try:
    accuracy, precision = calculate_accuracy_and_precision('actual_values.csv', 'predicted_values.csv')
    print(f'Accuracy: {accuracy:.2f}%')
    print(f'Precision: {precision:.2f}%')
except KeyError as e:
    print(f"KeyError: {e}. Please check the column names in your CSV files.")




#FOR THE FOLLOWING CSV FILE THE ACCURACY AND PRECISION CAN BE CALCULATED USING THE ABOVE CODE!